In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
%cd gdrive/MyDrive/colab_projects/recommendation_system/nbs/

/content/gdrive/MyDrive/colab_projects/recommendation_system/nbs


In [3]:
import sys
sys.path.append("../src/")
import os
import pandas as pd
import numpy as np
import tensorflow as tf
from config import *

In [4]:
pd.options.display.max_columns=999
pd.options.display.max_rows=999

In [5]:
df = pd.read_parquet("../data/processed/df_rating_movie.parquet")

In [6]:
df.head()

,userId,movieId,rating,timestamp,title,genres
0,1,296,5.0,1147880044,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller
1,1,306,3.5,1147868817,Three Colors: Red (Trois couleurs: Rouge) (1994),Drama
2,1,307,5.0,1147868828,Three Colors: Blue (Trois couleurs: Bleu) (1993),Drama
3,1,665,5.0,1147878820,Underground (1995),Comedy|Drama|War
4,1,899,3.5,1147868510,Singin' in the Rain (1952),Comedy|Musical|Romance


In [7]:
df.groupby(["movieId", "title", "genres"]).agg({"rating": "mean", "userId": "count"}).rename(columns={"userId": "n_reviews"}).sort_values("rating", ascending=False)

,,,rating,n_reviews
movieId,title,genres,,
136782,The Girl is in Trouble (2015),Thriller,5.0,1
186119,A Gift Horse (2015),Children,5.0,1
137032,The Perfect Neighbor (2005),Drama|Thriller,5.0,1
184643,Relentless (2018),Thriller,5.0,1
137038,The Perfect Wife (2001),Drama|Thriller,5.0,1
...,...,...,...,...
5805,Besotted (2001),Drama,0.5,2
199922,Bon Bini Holland 2 (2018),Comedy,0.5,1
194608,The Black Book (2018),Drama|Romance,0.5,1


In [29]:
df_top_review = df.groupby(["movieId", "title", "genres"]).agg({"rating": "mean", "userId": "count"}).rename(columns={"userId": "n_reviews"})
df_top_review = df_top_review[df_top_review.n_reviews > 10]
df_top_review.sort_values("rating", ascending=False)[:10]

,,,rating,n_reviews
movieId,title,genres,,
171011,Planet Earth II (2016),Documentary,4.483096,1124
159817,Planet Earth (2006),Documentary,4.464797,1747
318,"Shawshank Redemption, The (1994)",Crime|Drama,4.413576,81482
170705,Band of Brothers (2001),Action|Drama|War,4.398599,1356
158958,Pollyanna (2003),Children|Drama,4.384615,13
171331,The Work of Director Chris Cunningham (2003),(no genres listed),4.375000,12
169022,Classmates (2016),Animation,4.333333,12
171495,Cosmos,(no genres listed),4.326715,277
858,"Godfather, The (1972)",Crime|Drama,4.324336,52498


In [30]:
df_top_review.sort_values("n_reviews", ascending=False)[:10]

,,,rating,n_reviews
movieId,title,genres,,
356,Forrest Gump (1994),Comedy|Drama|Romance|War,4.048011,81491
318,"Shawshank Redemption, The (1994)",Crime|Drama,4.413576,81482
296,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller,4.188912,79672
593,"Silence of the Lambs, The (1991)",Crime|Horror|Thriller,4.151342,74127
2571,"Matrix, The (1999)",Action|Sci-Fi|Thriller,4.154099,72674
260,Star Wars: Episode IV - A New Hope (1977),Action|Adventure|Sci-Fi,4.120189,68717
480,Jurassic Park (1993),Action|Adventure|Sci-Fi|Thriller,3.679175,64144
527,Schindler's List (1993),Drama|War,4.247579,60411
110,Braveheart (1995),Action|Drama|War,4.002273,59184


In [23]:
df_top_review = df.groupby(["movieId", "title", "genres"]).agg({"rating": "mean", "userId": "count"}).rename(columns={"userId": "n_reviews"})
df_top_review = df_top_review[df_top_review.n_reviews > 20]
df_top_review.sort_values("rating", ascending=False)

,,,rating,n_reviews
movieId,title,genres,,
171011,Planet Earth II (2016),Documentary,4.483096,1124
159817,Planet Earth (2006),Documentary,4.464797,1747
318,"Shawshank Redemption, The (1994)",Crime|Drama,4.413576,81482
170705,Band of Brothers (2001),Action|Drama|War,4.398599,1356
171495,Cosmos,(no genres listed),4.326715,277
...,...,...,...,...
6483,From Justin to Kelly (2003),Musical|Romance,1.027578,417
120222,Foodfight! (2012),Action|Animation|Children|Comedy,1.000000,24
107704,Justin Bieber's Believe (2013),Documentary,0.904762,21


In [31]:
df_top_genres = df.groupby(["genres"]).agg({"rating": "mean", "userId": "count"}).rename(columns={"userId": "n_reviews"})
df_top_genres = df_top_genres[df_top_genres.n_reviews > 10]
df_top_genres.sort_values("rating", ascending=False)[:10]

,rating,n_reviews
genres,,
Action|Adventure|Mystery|Romance|Thriller,4.196617,16850
Film-Noir|Romance|Thriller,4.161201,5062
Action|Crime|Drama|Mystery|Sci-Fi|Thriller|IMAX,4.155508,38895
Adventure|Animation|Fantasy,4.153862,24132
Action|Adventure|Crime|Drama|Thriller,4.153175,20444
Film-Noir|Mystery,4.148480,12335
Animation|Children|Drama|Fantasy,4.148000,9375
Action|Crime|Drama|IMAX,4.147029,42189
Action|Animation|Crime|Sci-Fi,4.129717,212


In [32]:
df_top_genres.sort_values("n_reviews", ascending=False)[:10]

,rating,n_reviews
genres,,
Drama,3.692611,1765705
Comedy,3.249082,1512682
Comedy|Romance,3.373527,902051
Drama|Romance,3.668747,796335
Comedy|Drama,3.568055,721432
Comedy|Drama|Romance,3.588525,715769
Action|Adventure|Sci-Fi,3.554423,595361
Crime|Drama,3.991945,594888
Action|Crime|Thriller,3.522501,379990
